In [1]:
import requests
import json
import pandas as pd
import numpy as np
import re
from collections import Counter
from datetime import datetime, timezone
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta

### Get a user's playlists

In [9]:
def get_plst_ID(user_id, token, market = 'US', added_time = False):
    tidy = lambda s: s[17:]
    
    base_url = "https://api.spotify.com/v1/users/"
    query = f'{base_url}{user_id}/playlists'

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    plst_id = [tidy(x['uri'])for x in json_response['items']]
    if not added_time:
        return plst_id
    else:
        base_url = "https://api.spotify.com/v1/playlists/"
        add = []
        for idx in plst_id:
            query = f'{base_url}{idx}/tracks?market={market}'
            response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
            json_response = response.json()
            print(json_response['items'])
            add.append(json_response['items'][0]['added_at'])
        return(plst_id, add)

### Get a Playlist's Items (song_id and duration)

In [3]:
def get_ID(user_id, token, playlist_id, market = 'US'):
    tidy = lambda s: s[14:]
    base_url = "https://api.spotify.com/v1/playlists/"
    query = f'{base_url}{playlist_id}/tracks?market={market}'

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    return [tidy(x['track']['uri'])for x in json_response['items']], [json_response['items'][i]['track']['duration_ms'] for i in range(len(json_response['items']))]

### Get a user's songs from playlists

In [4]:
def convert_weight(time_list, half_decay = 1):
    ''' Calculate weights.'''
    ## I use exponential decay now.
    ## the weight decays to 50% every `half_decay` days
    today = datetime.now(timezone.utc)
    log_weights = np.array([- (today - parse(s)).days - (today - parse(s)).seconds / 86400 for s in time_list])
    log_weights -= np.max(log_weights)
    return np.exp(log_weights / half_decay * np.log(2))

def get_song_freq(user_id, token, market = 'US', start_plst = 0, end_plst = 20, decay_rate = 15):
    plst_ids, add = get_plst_ID(user_id, token, market, added_time = True)
    plst_ids, add = plst_ids[start_plst:(end_plst+1)], add[start_plst:(end_plst+1)]
    weights = convert_weight(add, decay_rate)
    ids = [get_ID(user_id, token, playlist, market)[0] for playlist in plst_ids]
    song_list = [(s, w) for ls, w in list(zip(ids, weights)) for s in ls]
    counts = {}
    for s, w in song_list:
        counts[s] = counts.get(s, 0) + w
    return(pd.DataFrame(sorted(counts.items(), key = lambda x: -x[1]), columns = ['song_id', 'weighted_freq']))

## Test

Note: token should have scope at least "playlist-modify-private", you can find get one at the botton GET TOKEN from [here](https://developer.spotify.com/console/get-recommendations/?limit=10&market=ES&seed_artists=4NHQUGzhtTLFvgF5SZesLK&seed_genres=classical%2Ccountry&seed_tracks=0c6xIDDpzE81m2q797ordA&min_acousticness=&max_acousticness=&target_acousticness=&min_danceability=&max_danceability=&target_danceability=&min_duration_ms=&max_duration_ms=&target_duration_ms=&min_energy=&max_energy=&target_energy=&min_instrumentalness=&max_instrumentalness=&target_instrumentalness=&min_key=&max_key=&target_key=&min_liveness=&max_liveness=&target_liveness=&min_loudness=&max_loudness=&target_loudness=&min_mode=&max_mode=&target_mode=&min_popularity=&max_popularity=&target_popularity=&min_speechiness=&max_speechiness=&target_speechiness=&min_tempo=&max_tempo=&target_tempo=&min_time_signature=&max_time_signature=&target_time_signature=&min_valence=&max_valence=&target_valence=).

User id can be obtained from [your Spotify profile](https://www.spotify.com/us/account/overview/?utm_source=spotify&utm_medium=menu&utm_campaign=your_account).

In [11]:
token = "BQAgrT_Dhb09KfbPqOg0NARlh2InO64fx-Gtudl2srDwqAS6gDEeqpl3QXkROxAGx1diSJXar_RnLTpBUUrW2BV0EFBvQBc90KFhCuXaIYZ1buOTSpsWiXcWkln52-VIuGLEUx5gOJGkS-TfG7C9029phC1p8FqLWMl0O_S7E4OABo9lAUPUzvQ6jIUE3w4F6soSFlD9kTnlIg"
user_id = "2272ss2bp3tznxg6k3v63kqxq"##"pbwppse1hilahmk43ls424ao4"
market = 'US'
start_plst = 0 ## 0 means start from the most recent one
end_plst = 10 ## 10 means end at the 10 recent one
## in this case, 11 playlists are used 

In [12]:
get_song_freq(user_id, token, market, start_plst, end_plst, decay_rate = 15)

[{'added_at': '2020-11-27T20:08:00Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/2272ss2bp3tznxg6k3v63kqxq'}, 'href': 'https://api.spotify.com/v1/users/2272ss2bp3tznxg6k3v63kqxq', 'id': '2272ss2bp3tznxg6k3v63kqxq', 'type': 'user', 'uri': 'spotify:user:2272ss2bp3tznxg6k3v63kqxq'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4mncDFjVLUa3s025Tct3Ry'}, 'href': 'https://api.spotify.com/v1/artists/4mncDFjVLUa3s025Tct3Ry', 'id': '4mncDFjVLUa3s025Tct3Ry', 'name': 'Claptone', 'type': 'artist', 'uri': 'spotify:artist:4mncDFjVLUa3s025Tct3Ry'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/4rXvRS91n7rdVrYUI3Riwu'}, 'href': 'https://api.spotify.com/v1/albums/4rXvRS91n7rdVrYUI3Riwu', 'id': '4rXvRS91n7rdVrYUI3Riwu', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b27372296030f87da7500d01e113', 'width': 640}, {'height'

[{'added_at': '2020-11-27T19:31:19Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/2272ss2bp3tznxg6k3v63kqxq'}, 'href': 'https://api.spotify.com/v1/users/2272ss2bp3tznxg6k3v63kqxq', 'id': '2272ss2bp3tznxg6k3v63kqxq', 'type': 'user', 'uri': 'spotify:user:2272ss2bp3tznxg6k3v63kqxq'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4mncDFjVLUa3s025Tct3Ry'}, 'href': 'https://api.spotify.com/v1/artists/4mncDFjVLUa3s025Tct3Ry', 'id': '4mncDFjVLUa3s025Tct3Ry', 'name': 'Claptone', 'type': 'artist', 'uri': 'spotify:artist:4mncDFjVLUa3s025Tct3Ry'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/4rXvRS91n7rdVrYUI3Riwu'}, 'href': 'https://api.spotify.com/v1/albums/4rXvRS91n7rdVrYUI3Riwu', 'id': '4rXvRS91n7rdVrYUI3Riwu', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b27372296030f87da7500d01e113', 'width': 640}, {'height'

[{'added_at': '2020-11-24T21:18:52Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/2272ss2bp3tznxg6k3v63kqxq'}, 'href': 'https://api.spotify.com/v1/users/2272ss2bp3tznxg6k3v63kqxq', 'id': '2272ss2bp3tznxg6k3v63kqxq', 'type': 'user', 'uri': 'spotify:user:2272ss2bp3tznxg6k3v63kqxq'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4mncDFjVLUa3s025Tct3Ry'}, 'href': 'https://api.spotify.com/v1/artists/4mncDFjVLUa3s025Tct3Ry', 'id': '4mncDFjVLUa3s025Tct3Ry', 'name': 'Claptone', 'type': 'artist', 'uri': 'spotify:artist:4mncDFjVLUa3s025Tct3Ry'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/4rXvRS91n7rdVrYUI3Riwu'}, 'href': 'https://api.spotify.com/v1/albums/4rXvRS91n7rdVrYUI3Riwu', 'id': '4rXvRS91n7rdVrYUI3Riwu', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b27372296030f87da7500d01e113', 'width': 640}, {'height'

[{'added_at': '2020-11-16T22:36:07Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/2272ss2bp3tznxg6k3v63kqxq'}, 'href': 'https://api.spotify.com/v1/users/2272ss2bp3tznxg6k3v63kqxq', 'id': '2272ss2bp3tznxg6k3v63kqxq', 'type': 'user', 'uri': 'spotify:user:2272ss2bp3tznxg6k3v63kqxq'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7uZckwZIIL89Paj8E1GEFO'}, 'href': 'https://api.spotify.com/v1/artists/7uZckwZIIL89Paj8E1GEFO', 'id': '7uZckwZIIL89Paj8E1GEFO', 'name': 'Teho', 'type': 'artist', 'uri': 'spotify:artist:7uZckwZIIL89Paj8E1GEFO'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/74EgL5CnIA1SeHkthfeQZ9'}, 'href': 'https://api.spotify.com/v1/albums/74EgL5CnIA1SeHkthfeQZ9', 'id': '74EgL5CnIA1SeHkthfeQZ9', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b27325dc400dcc239d68dc7cf29b', 'width': 640}, {'height': 30

[{'added_at': '2020-11-11T18:59:30Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/2272ss2bp3tznxg6k3v63kqxq'}, 'href': 'https://api.spotify.com/v1/users/2272ss2bp3tznxg6k3v63kqxq', 'id': '2272ss2bp3tznxg6k3v63kqxq', 'type': 'user', 'uri': 'spotify:user:2272ss2bp3tznxg6k3v63kqxq'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2orIEMw3OyKWIaZ6CGueGg'}, 'href': 'https://api.spotify.com/v1/artists/2orIEMw3OyKWIaZ6CGueGg', 'id': '2orIEMw3OyKWIaZ6CGueGg', 'name': 'The Cube Guys', 'type': 'artist', 'uri': 'spotify:artist:2orIEMw3OyKWIaZ6CGueGg'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/6wVAv9sK4cH6Keoe3XFjIq'}, 'href': 'https://api.spotify.com/v1/albums/6wVAv9sK4cH6Keoe3XFjIq', 'id': '6wVAv9sK4cH6Keoe3XFjIq', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273443e0c6e1599409d9e63e7b1', 'width': 640}, {'he

...

[{'added_at': '2020-10-16T18:51:08Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/2272ss2bp3tznxg6k3v63kqxq'}, 'href': 'https://api.spotify.com/v1/users/2272ss2bp3tznxg6k3v63kqxq', 'id': '2272ss2bp3tznxg6k3v63kqxq', 'type': 'user', 'uri': 'spotify:user:2272ss2bp3tznxg6k3v63kqxq'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4mncDFjVLUa3s025Tct3Ry'}, 'href': 'https://api.spotify.com/v1/artists/4mncDFjVLUa3s025Tct3Ry', 'id': '4mncDFjVLUa3s025Tct3Ry', 'name': 'Claptone', 'type': 'artist', 'uri': 'spotify:artist:4mncDFjVLUa3s025Tct3Ry'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/4rXvRS91n7rdVrYUI3Riwu'}, 'href': 'https://api.spotify.com/v1/albums/4rXvRS91n7rdVrYUI3Riwu', 'id': '4rXvRS91n7rdVrYUI3Riwu', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b27372296030f87da7500d01e113', 'width': 640}, {'height'

[{'added_at': '2020-10-14T18:01:07Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/2272ss2bp3tznxg6k3v63kqxq'}, 'href': 'https://api.spotify.com/v1/users/2272ss2bp3tznxg6k3v63kqxq', 'id': '2272ss2bp3tznxg6k3v63kqxq', 'type': 'user', 'uri': 'spotify:user:2272ss2bp3tznxg6k3v63kqxq'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3M1kdGJP7DlEBGPb1JtKJX'}, 'href': 'https://api.spotify.com/v1/artists/3M1kdGJP7DlEBGPb1JtKJX', 'id': '3M1kdGJP7DlEBGPb1JtKJX', 'name': 'Traf', 'type': 'artist', 'uri': 'spotify:artist:3M1kdGJP7DlEBGPb1JtKJX'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/5H74kTqy3Rgh4OYcgVpMBV'}, 'href': 'https://api.spotify.com/v1/albums/5H74kTqy3Rgh4OYcgVpMBV', 'id': '5H74kTqy3Rgh4OYcgVpMBV', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273900a8e36787701a4753ed863', 'width': 640}, {'height': 30

[{'added_at': '2020-10-02T21:04:12Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/2272ss2bp3tznxg6k3v63kqxq'}, 'href': 'https://api.spotify.com/v1/users/2272ss2bp3tznxg6k3v63kqxq', 'id': '2272ss2bp3tznxg6k3v63kqxq', 'type': 'user', 'uri': 'spotify:user:2272ss2bp3tznxg6k3v63kqxq'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'compilation', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'}, 'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of', 'id': '0LyfQWJT6nXafLPZqxe9Of', 'name': 'Various Artists', 'type': 'artist', 'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/1xH5Qac8t9MhUPa8o9zPz0'}, 'href': 'https://api.spotify.com/v1/albums/1xH5Qac8t9MhUPa8o9zPz0', 'id': '1xH5Qac8t9MhUPa8o9zPz0', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273410f0ca90dea8770ac8de7c3', 'width': 640

[{'added_at': '2020-10-01T20:12:26Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/2272ss2bp3tznxg6k3v63kqxq'}, 'href': 'https://api.spotify.com/v1/users/2272ss2bp3tznxg6k3v63kqxq', 'id': '2272ss2bp3tznxg6k3v63kqxq', 'type': 'user', 'uri': 'spotify:user:2272ss2bp3tznxg6k3v63kqxq'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6KcmUwBzfwLaYxdfIboqcp'}, 'href': 'https://api.spotify.com/v1/artists/6KcmUwBzfwLaYxdfIboqcp', 'id': '6KcmUwBzfwLaYxdfIboqcp', 'name': 'Kyla La Grange', 'type': 'artist', 'uri': 'spotify:artist:6KcmUwBzfwLaYxdfIboqcp'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/23fqKkggKUBHNkbKtXEls4'}, 'href': 'https://api.spotify.com/v1/artists/23fqKkggKUBHNkbKtXEls4', 'id': '23fqKkggKUBHNkbKtXEls4', 'name': 'Kygo', 'type': 'artist', 'uri': 'spotify:artist:23fqKkggKUBHNkbKtXEls4'}], 'external_urls': {'spotify': 'http

,song_id,weighted_freq
0,5z6iuIbnfNTJmhEL3dZwwl,4.770537
1,3dXl4Tiea3VLlacmKr5o3n,3.747556
2,6ggBsnmojOdKcbESrCnxUz,3.241922
3,4OOelW7GAFlzPbcVcnwEIo,3.196596
4,5Bh2co6rmQtagn427WElfj,3.109072
...,...,...
135,7w09r53aPON8ZUvEAnPe94,0.236776
136,4z7gh3aIZV9arbL9jJSc5J,0.236776
137,4Y2bR875LdPq9JILrY2FSw,0.236776
138,05daxiDLRI9gEy8xmtvgpv,0.236776
